<h1>Load Himawari-8 AHI data and generate true color RGB</h1>

In [1]:
from satpy.scene import Scene
from satpy.resample import get_area_def
from datetime import datetime

<p>We start by locating and specifying the AHI data to read:</p>

In [2]:
from glob import glob
FILES = glob("/home/a000680/data/himawari8/201502070300/*DAT")

<p>We load the data using the Himawari standard format reader</p>

In [3]:
scn = Scene(
      sensor='ahi',
      start_time=datetime(2015, 2, 7, 3, 0),
      end_time=datetime(2015, 2, 7, 3, 10),
      filenames=FILES,
      reader='ahi_hsd')

No handlers could be found for logger "satpy.readers"


<p>Here we do not use the high resolution data</p>

In [4]:
rgbname = 'true_color_reducedsize'

In [5]:
scn.load([rgbname])

In [6]:
scn.show(rgbname)

/home/a000680/usr/src/satpy/satpy/enhancements/__init__.py:51: RuntimeWarning: divide by zero encountered in log10
  np.log10(chn.data, out=chn.data)
/home/a000680/usr/src/satpy/satpy/enhancements/__init__.py:51: RuntimeWarning: invalid value encountered in log10
  np.log10(chn.data, out=chn.data)


<img src='./himawari_ahi_truecolor_201502070300_thumb.png'>

<img href='himawari_ahi_truecolor_201502070300.png'>

In [7]:
scn.save_dataset(rgbname, filename='himawari_ahi_truecolor_{datetime}.png'.format(datetime=scn.start_time.strftime('%Y%m%d%H%M')))

<p>Now let us define a map-projection and a sub area, and project the data on this area. We use Pyresample to define the area. This definition can also be put in the area.def or area.yaml configuration file</p>

In [8]:
from pyresample.utils import get_area_def
area_id = 'japan'
x_size = 2407
y_size = 1655
area_extent = (-1203877.326193321, -754860.2505908236, 1203877.3261933187, 900373.6358697552)
projection = '+proj=laea +lat_0=37.5 +lon_0=137.5 +ellps=WGS84'
description = "Japan"
proj_id = 'laea_137.5_37.5'
areadef = get_area_def(area_id, description, proj_id, projection,x_size, y_size, area_extent)

In [9]:
local_scene = scn.resample(areadef)

In [10]:
local_scene.show(rgbname)

<img src='./himawari_ahi_truecolor_201502070300_japan_thumb.png'>